# Chiptune Generator

Process
1. Imports ready-to-go dataset (created in data management)
2. Imports ready-to-go model(s)
3. Generates new chiptune music
4. Decodes into MIDI, saves it and plays it

## Imports

In [ ]:
# Packages

from pathlib import Path
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='pygame')

from funcs import *
from model_classes import *

In [ ]:
# File Paths
PROJECT_PATH = Path.cwd()
DATA_PATH = f"{PROJECT_PATH}music/cleaned_data/"  # must have 'train', 'test', and 'valid' subfolders

# Other variables
CIRCLE_OF_FIFTHS = [0, 7, 2, 9, 4, 11, 6, 1, 8, 3, 10, 5]
seq_len = 16

device = 'cuda' if torch.cuda.is_available() else 'cpu' # (testing with cpu)

In [ ]:
# Loading saved models

"""
CNN:
ConvAutoencoder(
    in_channels=1, feat_maps=(20,10), rows=24, cols=12, latent_dim=128
    )

LSTM:
SequencePredictor(
  (lstm): LSTM(128, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=288, bias=True)
  )
"""

autoenc = ConvAutoencoder(in_channels=1, feat_maps=(20,10), rows=24, cols=12, latent_dim=128)
autoenc.load_state_dict(torch.load(
    f"{PROJECT_PATH}models/autoencoder_final.pt", map_location=device))  # Fix: Add map_location for device compatibility
print("Loaded model from disk.")
print("Autoencoder:", autoenc.eval(), "\n")
seq_model = SequencePredictor(latent_dim=128, hidden_dim=256, num_layers=2, out_size=(1,24,12))
seq_model.load_state_dict(torch.load(
    f"{PROJECT_PATH}models/sequence_model_final.pt", map_location=device))  # Fix: Add map_location
print("Loaded model from disk.")
print("Sequence Predictor:", seq_model.eval(), "\n")


## Usage

The sequence generator is currently set up to generate a single sequence of notes based on a "seed" sequence.

In future versions, this process will be made seamless and feature a GUI with automatic playback.

In [ ]:
# 1) Set MIDI file location (for seed)
#-- for now, we will use a single MIDI file from the test set

test_seed = "286_Rygar_00_01StartingPointSunset_cleaned.mid"

test_data_path = f"{PROJECT_PATH}music/cleaned_data/test/"
test_seed_path = f"{test_data_path}{test_seed}"

In [ ]:
# Play test (optional)
play_midi(test_seed)

In [ ]:
# 2) Convert seed file to Tonnetz sequence
try:
    seq = midi_to_tonnetz_sequence(
                    test_seed_path,
                    rows=24,
                    cols=12,
                    quantize_beat=1.0
                )
    print(f"✓ Loaded {test_seed}. \nShape = {seq.shape}")
except Exception as e:
    print(f"✗ ERROR processing file:\033[0m")
    print(f"\033[0m", e)

In [ ]:
# 3) Generate music from seed

#-- Defining seed sequence
if len(seq) > 0:
    seed = seq[:seq_len]  # Fix: Slice first seq_len frames (shape: (16, 24, 12))
    print("Seed shape:", seed.shape)

#-- Generating music using seed (example: batch of 4, chained 2x for longer sequence, with probabilistic sampling)
    generated = generate_sequence(seed, autoenc, seq_model, gen_steps=64, device=device,
                                  sample_prob=True, num_chains=2, batch_size=4)
    print("Generated sequences shape:", generated.shape)  # e.g., (4, 16 + 64*2, 24, 12)

In [ ]:
# 4) Revert to MIDI format and save file

# Create subfolder in /generated/
output_dir = f"{PROJECT_PATH}/music/generated/{test_seed[:-4]}"
os.makedirs(output_dir, exist_ok=True)  # Create if it doesn't exist

# Save generated MIDI(s) in the new subfolder
if generated.ndim == 4 and generated.shape[0] > 1:  # Batched case
    for i in range(generated.shape[0]):
        pm = tonnetz_sequence_to_midi(generated[i])
        pm.write(f"{output_dir}/generated_{i}.mid")
else:  # Single sequence (flatten batch dim if needed)
    single_generated = generated[0] if generated.ndim == 4 else generated
    test_pm = tonnetz_sequence_to_midi(single_generated)
    test_pm.write(f"{output_dir}/generated.mid")

print(f"Generated MIDI(s) saved to: {output_dir}")

In [ ]:
# 5) Test playback
play_midi(f"{output_dir}/generated_0.mid", volume = 1.0)

In [ ]:
# 6) Graph piano roll (visualize notes)
generated_notes = midi_to_notes(f"{PROJECT_PATH}/music/generated/{test_seed[:-4]}/generated_0.mid")

plot_piano_roll(generated_notes, title="Piano Roll - Generated MIDI #1 (Rygar)")

In [ ]:
play_midi(f"{test_data_path}308_StarForce_01_02MainTheme_cleaned.mid")

In [ ]:
chiptune_generator("308_StarForce_01_02MainTheme_cleaned.mid")